In [ ]:
import os
from dotenv import load_dotenv
import json
import random
import matplotlib.pyplot as plt
import numpy as np
import asyncio

from benchmark import run_benchmark_sample, Trials, mark_job_failure, create_compilation_failure_trial

In [ ]:
# we are currently using include_existing_trials=True and slowly increasing the number of functions
# to gather data across multiple runs
# this allows us to get initial statistics across various complexities and number of variables before the entire benchmark is complete
# we keep num_inputs=100 since these will not get filled in in future runs
crashes = []
def run_benchmark(include_existing_trials=True):
    for complexity in range(8, 21):
        for num_vars in range(2, 16):
            if (num_vars, complexity) in crashes:
                print(f"Skipping {num_vars} vars and {complexity} complexity")
                continue
            # lowering shots to 10**3 to meet new system constraint (it is either that or reduce num_inputs / split into more batches)
            run_benchmark_sample(num_vars, complexity, num_functions=5, num_inputs=100, shots=10**3, include_existing_trials=include_existing_trials, circuits_per_job=25)



In [ ]:
run_benchmark()


In [ ]:
# run this cell to load the job results for all trials that are waiting pending job results
trials = Trials()
await trials.load_results()

In [ ]:
with trials._connect() as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM trials WHERE job_id = 'cz0ja91q2bag008ybrq0'")
    rows = cursor.fetchall()
    print(rows)

In [ ]:
# run for any statement that could not be compiled to a circuit
trials = Trials()
create_compilation_failure_trial(num_vars=15, complexity=18, statement="(((not (not (not (((((((((not ((not ((x3) and (x13))) or (not (x10)))) or (((x10) ^ (x9)) and (x11))) and (((x1) and (x8)) or (not (x14)))) or ((not ((x3) and (x13))) or (not (x10)))) and (not (not ((x4) and (x13))))) and ((((not ((not ((x3) and (x13))) or (not (x10)))) or (((x10) ^ (x9)) and (x11))) and (((x1) and (x8)) or (not (x14)))) ^ (((x1) and (x8)) or (not (x14))))) and (((not (((x4) and (x13)) and ((x7) or (x9)))) and ((x7) or (x9))) or (not (not (((x4) and (x13)) and ((x7) or (x9))))))) ^ ((((((x0) ^ (x10)) ^ (x2)) or (x0)) or (x11)) and ((not ((x1) and (x8))) or (not ((x1) and (x8)))))) or ((not ((((not ((x3) and (x13))) or (not (x10))) ^ (not (((x4) and (x13)) and ((x7) or (x9))))) and (not ((((x4) and (x13)) and ((x7) or (x9))) and (((x4) and (x13)) and ((x7) or (x9))))))) and ((x3) and (x13))))))) or (x4)) and (((x7) or (x9)) and ((x7) or (x9)))) or ((((((not (((not ((((not ((x3) and (x13))) or (not (x10))) ^ (((x4) and (x13)) and ((x7) or (x9)))) ^ ((((x0) ^ (x10)) ^ (x2)) or (x0)))) ^ (not (not (((not ((x1) and (x8))) ^ (not (x14))) ^ (x12))))) ^ (((((x1) ^ (x14)) ^ ((x9) or (x9))) or (x13)) ^ (x4)))) and ((((not ((not ((x3) and (x13))) or (not (x10)))) or (((x10) ^ (x9)) and (x11))) and (((x1) and (x8)) or (not (x14)))) ^ (((x1) and (x8)) or (not (x14))))) or ((x12) and (x4))) and (not (((((not ((x3) and (x13))) or (not (x10))) and (x7)) ^ (x14)) and ((x9) or (x9))))) or (x11)) ^ ((((x0) ^ (x10)) ^ (x2)) or (x0)))", trials=trials)

In [ ]:
# failures to execute the circuit
failed = []
for job_id in failed:
    mark_job_failure(job_id)

In [ ]:
# other failures (that should be removed from the circuit)
failed = []
for job_id in failed:
    with trials._connect() as conn:
        cursor = conn.cursor()
        cursor.execute("DELETE FROM trials WHERE job_id = ?", (job_id,))

In [ ]:
# example code to retrieve results from the first trial of 2 variables 2 complexity
trial = trials.get(num_vars=2, complexity=2)[0]
print("Experimental results:", trial.get_counts())
print("Expected:", trial.total_expected_results())
print("Number of matches:", trial.get_counts().get(trial.total_expected_results(), 0))
print("Shot count:", sum(trial.get_counts().values()))

from function_generator import get_python_function
print("Function used:", end="")
print(get_python_function(trial.statement, trial.variables))


In [ ]:
def get_probability_data(trials, function_success_threshold, use_hamming=False):
    num_vars_data = []
    complexity_data = []
    probability_data = []

    for num_vars in range(2, 33):
        for complexity in range(1, 22):
            print(f"(num_vars, complexity) = ({num_vars}, {complexity})")
    
            successful_function_count = 0
            trials_by_function = trials.get_per_statement(num_vars=num_vars, complexity=complexity)
            for function in trials_by_function.keys():
                s = 0.0
                for trial in trials_by_function[function]:
                    if use_hamming:
                        s += (1-trial.mean_hamming_distance)
                    else:
                        s += trial.exact_match_rate
                    
                if s / len(trials_by_function[function]) > function_success_threshold:
                    successful_function_count += 1

            if len(trials_by_function) == 0:
                 print(f"Warning: no results for {num_vars} variables, complexity {complexity}; skipping")
                 continue
            
            num_vars_data.append(num_vars)
            complexity_data.append(complexity)
            probability_data.append(successful_function_count / len(trials_by_function))
    
    return num_vars_data, complexity_data, probability_data
    

In [ ]:
trials = Trials()
num_vars_data_exact, complexity_data_exact, probability_data_exact = get_probability_data(trials, 0.8, use_hamming=False)

In [ ]:
trials = Trials()
num_vars_data_hamming, complexity_data_hamming, probability_data_hamming = get_probability_data(trials, 0.8, use_hamming=True)

In [ ]:

def plot_probability_data(num_vars_data, complexity_data, probability_data, title, threshold=None, filepath=None):
    if threshold is not None:
        colors = ['green' if p > threshold else 'red' for p in probability_data]
    else:
        colors = probability_data

    plt.figure(figsize=(20, 10))
    plt.scatter(num_vars_data, complexity_data, c=probability_data, cmap='RdYlGn', edgecolors='black', alpha=0.75, s=450)
    plt.xticks(np.arange(min(num_vars_data), max(num_vars_data) + 1, 1))
    plt.yticks(np.arange(min(complexity_data), max(complexity_data) + 1, 1))

    plt.xlabel('Variables Count')
    plt.ylabel('Complexity')
    plt.title(title)
    cbar = plt.colorbar()
    if filepath is not None:
        plt.savefig(filepath, dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
plot_probability_data(num_vars_data_exact, complexity_data_exact, probability_data_exact, 'Proportion of functions with exact match rate > 0.8', filepath='exact_match_rate.png')

In [ ]:
plot_probability_data(num_vars_data_hamming, complexity_data_hamming, probability_data_hamming, 'Proportion of functions with mean hamming distance < 0.2', filepath='hamming_distance.png')

In [ ]:
# plot the number of functions we have data for, for each number of variables and complexity
trials = Trials()
num_vars_data = []
complexity_data = []
function_count_data = []

for num_vars in range(2, 33):
        for complexity in range(1, 22):
            with trials._connect() as conn:
                cursor = conn.cursor()
                cursor.execute("SELECT COUNT(DISTINCT statement) FROM trials WHERE num_vars = ? AND complexity = ? AND NOT counts = ''", (num_vars, complexity))
                count = cursor.fetchone()[0]
                num_vars_data.append(num_vars)
                complexity_data.append(complexity)
                function_count_data.append(count)
            print(f"(num_vars, complexity) = ({num_vars}, {complexity})")



In [ ]:
for i in range(len(function_count_data)):
    if function_count_data[i] > 30:
        function_count_data[i] = 30

In [ ]:
def plot_counts_data(num_vars_data, complexity_data, probability_data, title, threshold=None, filepath=None):
    if threshold is not None:
        colors = ['green' if p > threshold else 'red' for p in probability_data]
    else:
        colors = probability_data

    plt.figure(figsize=(10, 10))
    plt.scatter(num_vars_data, complexity_data, c=probability_data, cmap='RdYlGn', edgecolors='black', alpha=0.75, s=250)
    plt.xticks(np.arange(min(num_vars_data), max(num_vars_data) + 1, 1))
    plt.yticks(np.arange(min(complexity_data), max(complexity_data) + 1, 1))

    plt.xlabel('Variables Count')
    plt.ylabel('Complexity')
    plt.title(title)
    cbar = plt.colorbar()
    if filepath is not None:
        plt.savefig(filepath, dpi=300, bbox_inches='tight')
    plt.show()
    
plot_counts_data(num_vars_data, complexity_data, function_count_data, 'Number of functions with data', filepath='function_count.png')

In [ ]:
partials = []
for i in range(len(function_count_data)):
    if function_count_data[i] > 0 and function_count_data[i] < 15:
        partials.append((num_vars_data[i], complexity_data[i]))
print(partials)